# Borda count Implementation

In [1]:
import pandas as pd

In [2]:
#voters are : accuracy,precision,recall and F1-score
#members are the classifiers
#the input is a dataframe of singles with the values of each voter 
def get_votes(data,voter):
    d = {"Model" : [], voter+"_rank" : []}
    # we sort the singles depending on the voter (ascending sort) 
    db = data[["Model",voter]].sort_values(by=voter)
    # The first choice of each voter got the highest rank = N (nb of classifiers) and the last choice got the lowest = 1
    rank = 1
    for model in db["Model"] :
      d['Model'].append(model)
      d[voter+"_rank"].append(rank)
      rank+=1
    res = pd.DataFrame(d)
    #return a dataset of the ranks depending on a spesific voter
    return res

In [3]:
#N : nb of models 
#results contains the ranking of the models by all the voters
def get_models_ranking(results,N) :
  #we put the models in index and the deferent ranks in the columns
  ranking = pd.DataFrame(index=[i for i in range(N)], columns = [j for j in range(1,N+1)])
  k = results.set_index("Model")
  for i in ranking.index :
    for j in ranking.columns :
      # we count the number of times the model i got the rank j
      if j in k.loc[i].value_counts().index :
        ranking[j][i] = k.loc[i].value_counts()[j]
      else :
        #if the model never had j as a rank we put 0 
        ranking[j][i] = 0
  return ranking

In [4]:
def Borda_count_score(data):
  #first let's get the votes from each voter
  Acc = get_votes(data,'accuracy')
  Pre = get_votes(data,'precision')
  recall = get_votes(data,'recall')
  F1_score = get_votes(data,'f1_score')
  Roc_auc = get_votes(data, "roc_auc")
  #we merge all the results 
  results = pd.merge(pd.merge(pd.merge(Acc,Pre, on="Model"),pd.merge(recall,F1_score, on="Model"), on="Model"), Roc_auc, on="Model")
  N = len(data)
  #we can now get the ranking matrix 
  ranking = get_models_ranking(results,N)
  final_res = {"Model" : [i for i in range(N)], "Borda_count_score" : []}
  for model in final_res["Model"] : 
    #for each model we calculate the borda count score 
    n=0
    for i in ranking.loc[model].index :
      # get the nb of times the model got the rank i times the value of the rank (i)
      n+= i*ranking.loc[model][i]
    final_res["Borda_count_score"].append(n)
  res = pd.DataFrame(final_res)
  #return the sorted values of the borda count scores
  return pd.merge(data,res,on="Model").sort_values(by="Borda_count_score",ascending=False)

# Borda count ranking for our different models

### Load the metrics of each models

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


##### Apply of the bordo count rank classment metric on tabular data


In [ ]:
path = "/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/tabular_model_metric.csv"
data= pd.read_csv(path)

In [ ]:
data.rename(columns = {'Unnamed: 0':'Model'}, inplace = True)
data

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc
0,0,SVC,Aucun,tabular_model,0.728113,0.926641,0.601986,0.728875,0.763216
1,1,KNN,Aucun,tabular_model,0.728491,0.832021,0.693575,0.756356,0.727272
2,2,DecisionTree,Aucun,tabular_model,0.736415,0.807698,0.743242,0.774037,0.727011
3,3,RandomForest,Aucun,tabular_model,0.741321,0.820573,0.735486,0.775632,0.733592
4,4,MLP,Aucun,tabular_model,0.737736,0.858987,0.680533,0.758763,0.742780


Borda count ranking for our different models

In [ ]:
result = Borda_count_score(data)

In [ ]:
result= result.drop(columns=["method","Model", "model_type"], axis=1)

In [ ]:
result

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
3,RandomForest,0.741321,0.820573,0.735486,0.775632,0.733592,19
4,MLP,0.737736,0.858987,0.680533,0.758763,0.742780,17
2,DecisionTree,0.736415,0.807698,0.743242,0.774037,0.727011,14
0,SVC,0.728113,0.926641,0.601986,0.728875,0.763216,13
1,KNN,0.728491,0.832021,0.693575,0.756356,0.727272,12


In [ ]:
result.to_csv("/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Result_borda_count_tabular_data.csv")

##### Apply of the bordo count rank classment metric on deep models

In [ ]:
path = "/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Deep_model_metric.csv"
data= pd.read_csv(path)

In [ ]:
data.rename(columns = {'Unnamed: 0':'Model'}, inplace = True)
data

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc
0,0,VGG16,Aucun,Deep_model,0.668679,0.688592,0.832965,0.753193,0.653881
1,1,VGG19,Aucun,Deep_model,0.662642,0.680320,0.849104,0.752909,0.654945
2,2,ResNet50,Aucun,Deep_model,0.655660,0.682451,0.816837,0.741763,0.639332
3,3,InceptionResNetV2,Aucun,Deep_model,0.635660,0.655843,0.857464,0.739594,0.623081
4,4,InceptionV3,Aucun,Deep_model,0.635283,0.655796,0.850021,0.738192,0.615988
5,5,Xception,Aucun,Deep_model,0.634906,0.656420,0.840124,0.736257,0.611254
6,6,DenseNet201,Aucun,Deep_model,0.766981,0.799765,0.823031,0.811039,0.756475
7,7,MobileNet,Aucun,Deep_model,0.654906,0.672705,0.846012,0.748262,0.639870
8,8,NASNetMobile,Aucun,Deep_model,0.648679,0.675733,0.818050,0.737740,0.633224


In [ ]:
result= Borda_count_score(data)

In [ ]:
result

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
6,6,DenseNet201,Aucun,Deep_model,0.766981,0.799765,0.823031,0.811039,0.756475,39
0,0,VGG16,Aucun,Deep_model,0.668679,0.688592,0.832965,0.753193,0.653881,35
1,1,VGG19,Aucun,Deep_model,0.662642,0.680320,0.849104,0.752909,0.654945,35
7,7,MobileNet,Aucun,Deep_model,0.654906,0.672705,0.846012,0.748262,0.639870,27
2,2,ResNet50,Aucun,Deep_model,0.655660,0.682451,0.816837,0.741763,0.639332,24
3,3,InceptionResNetV2,Aucun,Deep_model,0.635660,0.655843,0.857464,0.739594,0.623081,21
8,8,NASNetMobile,Aucun,Deep_model,0.648679,0.675733,0.818050,0.737740,0.633224,17
4,4,InceptionV3,Aucun,Deep_model,0.635283,0.655796,0.850021,0.738192,0.615988,16
5,5,Xception,Aucun,Deep_model,0.634906,0.656420,0.840124,0.736257,0.611254,11


In [ ]:
result= result.drop(columns=["method","Model", "model_type"], axis=1)

In [ ]:
result

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
6,DenseNet201,0.766981,0.799765,0.823031,0.811039,0.756475,39
0,VGG16,0.668679,0.688592,0.832965,0.753193,0.653881,35
1,VGG19,0.662642,0.680320,0.849104,0.752909,0.654945,35
7,MobileNet,0.654906,0.672705,0.846012,0.748262,0.639870,27
2,ResNet50,0.655660,0.682451,0.816837,0.741763,0.639332,24
3,InceptionResNetV2,0.635660,0.655843,0.857464,0.739594,0.623081,21
8,NASNetMobile,0.648679,0.675733,0.818050,0.737740,0.633224,17
4,InceptionV3,0.635283,0.655796,0.850021,0.738192,0.615988,16
5,Xception,0.634906,0.656420,0.840124,0.736257,0.611254,11


In [ ]:
result.to_csv("/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Result_borda_count_deep_model_data.csv")

#### Apply of the bordo count rank classment metric on fusion models

In [6]:
path = "/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Fusion_model_metric.csv"
data= pd.read_csv(path)

In [7]:
data.rename(columns = {'Unnamed: 0':'Model'}, inplace = True)
data

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc
0,0,VGG16+SVC+avg,avg,Fusion,0.726981,0.724985,0.889162,0.798347,0.730900
1,1,VGG16+KNN+avg,avg,Fusion,0.737547,0.736893,0.885128,0.803943,0.739374
2,2,VGG16+DecisionTree+avg,avg,Fusion,0.732264,0.732399,0.883264,0.800448,0.733572
3,3,VGG16+RandomForest+avg,avg,Fusion,0.739245,0.735713,0.892579,0.806259,0.744158
4,4,VGG16+MLP+avg,avg,Fusion,0.737358,0.736220,0.887301,0.804283,0.740126
...,...,...,...,...,...,...,...,...,...
130,130,NASNetMobile+SVC+sum,sum,Fusion,0.683396,0.693523,0.865552,0.768231,0.682571
131,131,NASNetMobile+KNN+sum,sum,Fusion,0.714528,0.708547,0.907785,0.794746,0.729039
132,132,NASNetMobile+DecisionTree+sum,sum,Fusion,0.715283,0.709861,0.905301,0.794699,0.728415
133,133,NASNetMobile+RandomForest+sum,sum,Fusion,0.707547,0.704080,0.901884,0.789536,0.720451


In [8]:
data_max=data[data["method"]=='max']
data_sum=data[data["method"]=='sum']
data_avg=data[data["method"]=='avg']

In [9]:
Fusion_borda = Borda_count_score(data)

In [10]:
Fusion_borda= Fusion_borda.drop(columns=["method","Model", "model_type"], axis=1)

In [11]:
Fusion_borda

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
31,DenseNet201+KNN+avg,0.821509,0.820881,0.904065,0.860287,0.822546,615
76,DenseNet201+KNN+max,0.820000,0.820485,0.901581,0.858934,0.820505,605
121,DenseNet201+KNN+sum,0.820000,0.820485,0.901581,0.858934,0.820505,604
32,DenseNet201+DecisionTree+avg,0.820000,0.821249,0.900031,0.858728,0.819785,602
77,DenseNet201+DecisionTree+max,0.818679,0.821099,0.897547,0.857504,0.817930,593
...,...,...,...,...,...,...,...
60,InceptionResNetV2+SVC+max,0.661698,0.668540,0.892864,0.761941,0.669564,66
100,ResNet50+SVC+sum,0.671509,0.689778,0.840743,0.756315,0.660368,65
20,InceptionV3+SVC+avg,0.663019,0.668782,0.890699,0.762454,0.663203,64
65,InceptionV3+SVC+max,0.663019,0.668782,0.890699,0.762454,0.663203,61


In [12]:
Fusion_borda.to_csv("/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Result_borda_count_Fusion_all.csv")

#### Max fusion only borda count

In [44]:
data_sum

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc
90,90,VGG16+SVC+sum,sum,Fusion,0.726981,0.724985,0.889162,0.798347,0.730900
91,91,VGG16+KNN+sum,sum,Fusion,0.737547,0.736893,0.885128,0.803943,0.739374
92,92,VGG16+DecisionTree+sum,sum,Fusion,0.732264,0.732399,0.883264,0.800448,0.733572
93,93,VGG16+RandomForest+sum,sum,Fusion,0.739245,0.735713,0.892579,0.806259,0.744158
94,94,VGG16+MLP+sum,sum,Fusion,0.737358,0.736220,0.887301,0.804283,0.740126
95,95,VGG19+SVC+sum,sum,Fusion,0.722642,0.719256,0.898160,0.797449,0.735374
96,96,VGG19+KNN+sum,sum,Fusion,0.734151,0.731882,0.893503,0.803465,0.742097
97,97,VGG19+DecisionTree+sum,sum,Fusion,0.730755,0.728665,0.893815,0.801503,0.739617
98,98,VGG19+RandomForest+sum,sum,Fusion,0.730755,0.727671,0.896608,0.802068,0.740534
99,99,VGG19+MLP+sum,sum,Fusion,0.730189,0.728050,0.894435,0.801320,0.739545


In [45]:
data_sum=data_sum.drop(columns=["method","Model", "model_type"], axis=1)

In [46]:
max=range(data_sum.shape[0])
data_sum['Model']=max

In [47]:
data_sum

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Model
90,VGG16+SVC+sum,0.726981,0.724985,0.889162,0.798347,0.730900,0
91,VGG16+KNN+sum,0.737547,0.736893,0.885128,0.803943,0.739374,1
92,VGG16+DecisionTree+sum,0.732264,0.732399,0.883264,0.800448,0.733572,2
93,VGG16+RandomForest+sum,0.739245,0.735713,0.892579,0.806259,0.744158,3
94,VGG16+MLP+sum,0.737358,0.736220,0.887301,0.804283,0.740126,4
95,VGG19+SVC+sum,0.722642,0.719256,0.898160,0.797449,0.735374,5
96,VGG19+KNN+sum,0.734151,0.731882,0.893503,0.803465,0.742097,6
97,VGG19+DecisionTree+sum,0.730755,0.728665,0.893815,0.801503,0.739617,7
98,VGG19+RandomForest+sum,0.730755,0.727671,0.896608,0.802068,0.740534,8
99,VGG19+MLP+sum,0.730189,0.728050,0.894435,0.801320,0.739545,9


In [48]:
data_sum= Borda_count_score(data_sum)
X_max=data_sum.head(4)

In [49]:
X_max= X_max.drop(columns=["Model","Borda_count_score"], axis=1)
max=range(X_max.shape[0])
X_max['Model']=max
X_max=Borda_count_score(X_max)
X_max

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Model,Borda_count_score
0,DenseNet201+KNN+sum,0.820000,0.820485,0.901581,0.858934,0.820505,0,19
1,DenseNet201+DecisionTree+sum,0.818679,0.821099,0.897547,0.857504,0.817930,1,16
2,DenseNet201+MLP+sum,0.815660,0.818498,0.895686,0.855205,0.814839,2,9
3,DenseNet201+RandomForest+sum,0.814528,0.818722,0.892889,0.854046,0.813070,3,6


In [50]:
X_max=X_max.drop(columns=["Model"], axis=1)
X_max

,Model_name,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
0,DenseNet201+KNN+sum,0.820000,0.820485,0.901581,0.858934,0.820505,19
1,DenseNet201+DecisionTree+sum,0.818679,0.821099,0.897547,0.857504,0.817930,16
2,DenseNet201+MLP+sum,0.815660,0.818498,0.895686,0.855205,0.814839,9
3,DenseNet201+RandomForest+sum,0.814528,0.818722,0.892889,0.854046,0.813070,6


In [ ]:
X_max.to_csv("/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/Result_borda_count_Fusion_sum_best.csv")

#### Average fusion best borda

##### Apply of the bordo count rank classment metric on deep models

In [ ]:
path = "/content/drive/MyDrive/Data_PS3/Save_dir/result/Final_metric/GLobal_metric.csv"
data= pd.read_csv(path)

In [ ]:
data.rename(columns = {'Unnamed: 0':'Model'}, inplace = True)
data

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc
0,0,VGG16+SVC+avg,avg,Fusion,0.726981,0.724985,0.889162,0.798347,0.730900
1,1,VGG16+KNN+avg,avg,Fusion,0.737547,0.736893,0.885128,0.803943,0.739374
2,2,VGG16+DecisionTree+avg,avg,Fusion,0.732264,0.732399,0.883264,0.800448,0.733572
3,3,VGG16+RandomForest+avg,avg,Fusion,0.739245,0.735713,0.892579,0.806259,0.744158
4,4,VGG16+MLP+avg,avg,Fusion,0.737358,0.736220,0.887301,0.804283,0.740126
...,...,...,...,...,...,...,...,...,...
144,144,SVC,Aucun,tabular_model,0.728113,0.926641,0.601986,0.728875,0.763216
145,145,KNN,Aucun,tabular_model,0.728491,0.832021,0.693575,0.756356,0.727272
146,146,DecisionTree,Aucun,tabular_model,0.736415,0.807698,0.743242,0.774037,0.727011
147,147,RandomForest,Aucun,tabular_model,0.741321,0.820573,0.735486,0.775632,0.733592


In [ ]:
Borda_count_score(data)

,Model,Model_name,method,model_type,accuracy,recall,precision,f1_score,roc_auc,Borda_count_score
31,31,DenseNet201+KNN+avg,avg,Fusion,0.821509,0.820881,0.904065,0.860287,0.822546,682
121,121,DenseNet201+KNN+sum,sum,Fusion,0.820000,0.820485,0.901581,0.858934,0.820505,671
32,32,DenseNet201+DecisionTree+avg,avg,Fusion,0.820000,0.821249,0.900031,0.858728,0.819785,670
76,76,DenseNet201+KNN+max,max,Fusion,0.820000,0.820485,0.901581,0.858934,0.820505,669
77,77,DenseNet201+DecisionTree+max,max,Fusion,0.818679,0.821099,0.897547,0.857504,0.817930,659
...,...,...,...,...,...,...,...,...,...,...
142,142,MobileNet,Aucun,Deep_model,0.654906,0.672705,0.846012,0.748262,0.639870,42
138,138,InceptionResNetV2,Aucun,Deep_model,0.635660,0.655843,0.857464,0.739594,0.623081,33
143,143,NASNetMobile,Aucun,Deep_model,0.648679,0.675733,0.818050,0.737740,0.633224,32
139,139,InceptionV3,Aucun,Deep_model,0.635283,0.655796,0.850021,0.738192,0.615988,28
